Exercise 1: Bank Account
Instructions

Part I:

    Create a class called BankAccount that contains the following attributes and methods:
        balance - (an attribute)
        __init__ : initialize the attribute
        deposit : - (a method) accepts a positive int and adds to the balance, raise an Exception if the int is not positive.
        withdraw : - (a method) accepts a positive int and deducts from the balance, raise an Exception if not positive


Part II : Minimum balance account

    Create a MinimumBalanceAccount that inherits from BankAccount.
    Extend the __init__ method and accept a parameter called minimum_balance with a default value of 0.
    Override the withdraw method so it only allows the user to withdraw money if the balance remains higher than the minimum_balance, raise an Exception if not.


Part III: Expand the bank account class

    Add the following attributes to the BankAccount class:
        username
        password
        authenticated (False by default)

    Create a method called authenticate. This method should accept 2 strings : a username and a password. If the username and password match the attributes username and password the method should set the authenticated boolean to True.

    Edit withdraw and deposit to only work if authenticated is set to True, if someone tries an action without being authenticated raise an Exception


Part IV: BONUS Create an ATM class

    __init__:
        Accepts the following parameters: account_list and try_limit.

        Validates that account_list contains a list of BankAccount or MinimumBalanceAccount instances.
        Hint: isinstance()

        Validates that try_limit is a positive number, if you get an invalid input raise an Exception, then move along and set try_limit to 2.
        Hint: Check out this tutorial

        Sets attribute current_tries = 0

        Call the method show_main_menu (see below)

    Methods:
        show_main_menu:
            This method will start a while loop to display a menu letting a user select:
                Log in : Will ask for the users username and password and call the log_in method with the username and password (see below).
                Exit.

        log_in:
            Accepts a username and a password.

            Checks the username and the password against all accounts in account_list.
                If there is a match (ie. use the authenticate method), call the method show_account_menu.
                If there is no match with any existing accounts, increment the current tries by 1. Continue asking the user for a username and a password, until the limit is reached (ie. try_limit attribute). Once reached display a message saying they reached max tries and shutdown the program.

        show_account_menu:
            Accepts an instance of BankAccount or MinimumBalanceAccount.
            The method will start a loop giving the user the option to deposit, withdraw or exit.


In [5]:
class BankAccount:
    #Compte bancaire
    def __init__(self,balance=0,username="",password=""):
        self.balance=balance
        self.username=username
        self._password=password
        self.authenticated=False

    def authenticate(self,username,password):
        #auth.
        self.authenticated=(username==self.username and password==self._password)
        return self.authenticated

    def _check_auth(self):
        if not self.authenticated:
            raise PermissionError("Non authentifié")

    def deposit(self,amount):
        self._check_auth()
        if amount<=0:
            raise ValueError("Montant dépôt négatif")
        self.balance+=amount

    def withdraw(self,amount):
        self._check_auth()
        if amount<=0:
            raise ValueError("Montant retrait négatif")
        self.balance-=amount


class MinimumBalanceAccount(BankAccount):
    #Compte bancaire avec solde minimum
    def __init__(self,balance=0,username="",password="",minimum_balance=0):
        super().__init__(balance,username,password)
        self.minimum_balance=minimum_balance

    def withdraw(self,amount):
        self._check_auth()
        if amount<=0:
            raise ValueError("Montant retrait négatif")
        if self.balance-amount<self.minimum_balance:
            raise ValueError("Sous le minimum autorisé")
        self.balance-=amount


class ATM:
    #Distributeur
    def __init__(self,accounts,try_limit=2):
        if not all(isinstance(a,BankAccount) for a in accounts):
            raise TypeError("accounts doit contenir des BankAccount")
        if try_limit<=0:
            print("try_limit invalide, défaut 2")
            try_limit=2
        self.accounts=accounts
        self.try_limit=try_limit
        self.tries=0
        self.show_main_menu()

    def show_main_menu(self):
        while True:
            print("\n--- ATM ---\n1.Connexion\n2.Sortir")
            match input("Choix: "):
                case "1":
                    self._handle_login()
                case "2":
                    print("Au revoir");break
                case _:
                    print("Choix invalide")

    def _handle_login(self):
        user=input("Username: ")
        pwd=input("Password: ")
        for acc in self.accounts:
            if acc.authenticate(user,pwd):
                print(f"Bienvenue {user}")
                self.tries=0
                self.show_account_menu(acc)
                return
        self.tries+=1
        print("Identifiants incorrects")
        if self.tries>=self.try_limit:
            print("Trop d’essais");exit()

    def show_account_menu(self,acc):
        while True:
            print(f"\nSolde: {acc.balance}")
            print("1.Depot\n2.Retrait\n3.Deconnexion")
            match input("Choice: "):
                case "1":
                    self._handle_transaction(acc,acc.deposit,"dépôt")
                case "2":
                    self._handle_transaction(acc,acc.withdraw,"retrait")
                case "3":
                    acc.authenticated=False
                    print("Déconnexion")
                    break
                case _:
                    print("Choix invalide")

    @staticmethod
    def _handle_transaction(acc,operation,label):
        try:
            amt=int(input(f"Montant {label}: "))
            operation(amt)
            print(f"{label.capitalize()} effectué")
        except ValueError as e:
            print(f"Erreur: {e}")
        except PermissionError as e:
            print(f"Erreur: {e}")


if __name__ == "__main__":
    acc1=BankAccount(500,"alice","1234")
    acc2=MinimumBalanceAccount(1000,"bob","abcd",minimum_balance=100)
    ATM([acc1,acc2],try_limit=3)


--- ATM ---
1.Connexion
2.Sortir
Au revoir
